# Assignment 3
Jithin Eapen (CWID - 893390096)

In [10]:
import numpy as np

```
Computational Graph for a single Perceptron
x1 -----\
         \
          \
          
          sig(w1 * x1 + w2 * x2 + b) ---- O (L)
          /
         /
x2 -----/        
```

here `O` is the output of the perceptron and `L` is the loss.

## Ref: Sigmoid Gradient
$$\sigma(x) = \frac{1}{1+e^{-x}}$$

$$\frac{\delta \sigma(x)}{\delta x} = \sigma(x)(1 - \sigma(x))$$

NOTE: Other gradients can be easily calculated

---
# Generic class definitions for various operators/nodes

In [11]:
class Node(object):
    """
    Base object for all inputs and outputs.
    """
    def __init__(self, value, grad):
        self.value = value
        self.gradient = grad

In [12]:
class MultiplyNode(object):
    """
    Multiplies two inputs
    """
    def forward(self, x1, x2):
        self.x1 = x1
        self.x2 = x2
        self.output = Node(self.x1.value * self.x2.value, 0)
        return self.output
    
    def backward(self):
        self.x1.gradient = self.x2.value * self.output.gradient
        self.x2.gradient = self.x1.value * self.output.gradient

In [13]:
class AddNode(object):    
    """
    Adds two inputs x1 and x2.
    """
    def forward(self, x1, x2):
        self.x1 = x1
        self.x2 = x2
        self.output = Node(self.x1.value + self.x2.value, 0)
        return self.output
    
    def backward(self):
        self.x1.gradient = 1 * self.output.gradient
        self.x2.gradient = 1 * self.output.gradient

In [14]:
class SigmoidNode(object):
    """
    Adds a sigmoid non-linearity to a single input
    """
    def forward(self, x):
        self.x = x
        self.output = Node(1/(1 + np.exp(-1 * self.x.value)), 0.0)
        return self.output
        
    def backward(self):
        s = 1/(1 + np.exp(-1 * self.x.value))
        self.x.gradient = (s * (1 - s)) * self.output.gradient

---
# Putting it all together into a single Perceptron

In [15]:
class Perceptron(object):
    def __init__(self, x1, x2, alpha=0.001):
        ### Hyper parameters
        self.alpha = alpha
        ### Initializing weights/bias to a random float between -1 and 1.
        self.w1 = Node(np.random.uniform(-1, 1), 0.0)
        self.w2 = Node(np.random.uniform(-1, 1), 0.0)
        self.b = Node(np.random.uniform(-1, 1), 0.0)
        ### Input and Output variables
        self.x1 = Node(x1, 0.0)
        self.x2 = Node(x2, 0.0)
        ### Initialize operators nodes required 
        ### for processing the inputs within a perceptron
        self.initialize_operators()
    
    def initialize_operators(self):
        self.w1_mul_x1 = MultiplyNode()
        self.w2_mul_x2 = MultiplyNode()
        self.w1x1_add_w2x2 = AddNode()
        self.w1x1_w2x2_add_b = AddNode()
        self.sigmoid = SigmoidNode()
    
    def forward(self):
        w1x1 = self.w1_mul_x1.forward(self.w1, self.x1)
        w2x2 = self.w2_mul_x2.forward(self.w2, self.x2)
        w1x1_w2x2 = self.w1x1_add_w2x2.forward(w1x1, w2x2)
        w1x1_w2x2_b = self.w1x1_w2x2_add_b.forward(w1x1_w2x2, self.b)
        self.sigmoid.forward(w1x1_w2x2_b)
        
    def backward(self):
        self.sigmoid.backward()
        self.w1x1_w2x2_add_b.backward()
        self.w1x1_add_w2x2.backward()
        self.w2_mul_x2.backward()
        self.w1_mul_x1.backward()
    
    def updateWeightsAndBias(self):
        self.w1.value -= self.alpha * self.w1.gradient
        self.w2.value -= self.alpha * self.w2.gradient
        self.b.value -= self.alpha * self.b.gradient
        
    # For updating the inputs of the second perceptron
    def updateInputs(self):
        self.x1.value -= self.alpha * self.x1.gradient
        self.x2.value -= self.alpha * self.x2.gradient        

---

# Assignment 3

```
X1 ------\
          \    X3
          (P) ---- (P) ---- (T)
          /        /
X2 ------/        /
X4---------------/
```

1. Create a two layer neural network with one perceptron in each layer (see Diagram above). Write a validation code that does along with your implementation. The goal of network is to optimize the two perceptrons to produce the output target `T` given the inputs `X1` and `X2`. Assume the output `O` of each perceptron is

$$ O = \sigma{(w1*x1 + w2*x2 + b)} $$ where

$$\sigma(x) = \frac{1}{1+e^{-x}}$$

Feel free to change the loss function if you like.



# Validation for double layer perceptrons

In [23]:
#Output Target
T = 0.8113

#Inputs for first Perceptron
x1,x2 = (0.1, 0.23)

#Dummy Inputs for second Perceptron, which will be learned
x3,x4 = (0.112, 0.37)

#Hyper parameters
alpha = 0.1

#Number of iterations
N = 100000

p1 = Perceptron(x1, x2, alpha = 0.1)
p2 = Perceptron(x3, x4, alpha = 0.1)

for i in range (N):
    p1.forward();

    #Give forward output of first Perceptron as input to second Perceptron    
    p2.x1 = Node(p1.sigmoid.output.value, 0.0)

    p2.forward()

    p2.sigmoid.output.gradient = -2 * (T - p2.sigmoid.output.value)

    p2.backward()

    # Update both weights and inputs for 2nd perceptron
    p2.updateWeightsAndBias()
    p2.updateInputs()

    #Changed input of 2nd perceptron is the target value for 1st perceptron
    p1_target = p2.x1.value

    p1.sigmoid.output.gradient = -2 * (p1_target - p1.sigmoid.output.value)

    p1.backward()

    # Update for 1st perceptron, only weights
    p1.updateWeightsAndBias()

    

print "Expected output :" + str(T) + "\tActual output :" + str(p2.sigmoid.output.value)
print "\nPerceptron 1 static inputs :" + str(p1.x1.value) + " and " + str(p1.x2.value)
print "Perceptron 2 changed inputs :" + str(p2.x1.value) + " and " + str(p2.x2.value)

Expected output :0.8113	Actual output :0.8113

Perceptron 1 static inputs :0.1 and 0.23
Perceptron 2 changed inputs :0.678376881237 and 0.639712580263
